## Vegetation indexes to support the burned area delineation 


In [1]:
workflow = dict([('id', 'vegetation-index'),
                ('label', 'Vegetation index'),
                ('doc', 'Vegetation index processor')])


In [2]:
input_reference = dict([('id', 'input_reference'), 
                        ('label', 'EO product for vegetation index'),
                        ('doc', 'EO product for vegetation index'),
                        ('value', '/workspace/ogc-tb16/vegetation-index/t0yum2_z'), 
                        ('type', 'Directory'),
                        ('scatter', 'True')])

In [3]:
aoi = dict([('id', 'aoi'), 
              ('label', 'Area of interest'),
              ('doc', 'Area of interest in WKT'),
              ('value', 'POLYGON((136.508 -36.108,136.508 -35.654,137.178 -35.654,137.178 -36.108,136.508 -36.108))'), 
              ('type', 'string')])

### Vegetation indexes

NBR = (NIR - SWIR22) / (NIR + SWIR22)
NDVI = (NIR - Red) / (NIR + Red)
NDWI = (NIR - SWIR16) / (NIR + SWIR16)

In [6]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from shapely.wkt import loads
from time import sleep
from helpers import get_item, cog, set_env, get_assets, normalized_difference

set_env()

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
item = get_item(os.path.join(input_reference['value'], 
                                 'catalog.json'))

In [9]:
item

<Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>

In [10]:
asset_red, asset_nir, asset_swir16, asset_swir22 = get_assets(item)

In [11]:
vrt = '{0}.vrt'.format(item.id)
    
ds = gdal.BuildVRT(vrt,
               [asset_red, asset_nir, asset_swir16, asset_swir22],
               srcNodata=0,
               xRes=10, 
               yRes=10,
               separate=True)

ds.FlushCache()

ds = None

del(ds)

In [12]:
tif = '{0}.tif'.format(item.id)
    
min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

gdal.Translate(tif,
               vrt,
               outputType=gdal.GDT_Int16,
               projWin=[min_lon, max_lat, max_lon, min_lat],
               projWinSRS='EPSG:4326')

os.remove(vrt)

In [13]:
ds = gdal.Open(tif)

width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

red = ds.GetRasterBand(1).ReadAsArray()
nir = ds.GetRasterBand(2).ReadAsArray()
swir16 = ds.GetRasterBand(3).ReadAsArray()
swir22 = ds.GetRasterBand(4).ReadAsArray()

ds = None

os.remove(tif)

del(ds)

In [15]:
nbr = normalized_difference(nir, swir22)
    
swir22 = None

del(swir22)

ndvi = normalized_difference(nir, red)

red = None

del(red)

ndwi = normalized_difference(nir, swir16)

nir = swir16 = None

del(nir)

del(swir16)

In [16]:
catalog = Catalog(id='catalog-{}'.format(item.id),
                      description='Results')

catalog.clear_items()
catalog.clear_children()

item_name = 'INDEX_{}'.format(item.id)


item.properties.pop('eo:bands', None)
item.properties['eo:gsd'] = 10
item.properties['eo:platform'] = item.properties['platform']
item.properties['eo:instrument'] = 'MSI'

result_item = Item(id=item_name,
                   geometry=item.geometry,
                   bbox=item.bbox,
                   datetime=item.datetime,
                   properties=item.properties)

result_item.common_metadata.set_gsd(10)

eo_item = extensions.eo.EOItemExt(result_item)

In [18]:
default_bands = [{'name': 'NBR',
              'common_name': 'nbr'}, 
             {'name': 'NDVI',
              'common_name': 'ndvi'},
             {'name': 'NDWI',
              'common_name': 'ndwi'}]

In [19]:
bands = []
    
for index, veg_index in enumerate(['NBR', 'NDVI', 'NDWI']):

    temp_name = '_{}_{}.tif'.format(veg_index, item.id)
    output_name = '{}_{}.tif'.format(veg_index, item.id)

    driver = gdal.GetDriverByName('GTiff')

    output = driver.Create(temp_name, 
                           width, 
                           height, 
                           1, 
                           gdal.GDT_Int16)

    output.SetGeoTransform(input_geotransform)
    output.SetProjection(input_georef)
    output.GetRasterBand(1).WriteArray(nbr),

    output.FlushCache()

    sleep(5)

    output = None

    del(output)

    os.makedirs(os.path.join('.', item_name),
                exist_ok=True)

    cog(temp_name, os.path.join('.', item_name, output_name))

    result_item.add_asset(key=veg_index.lower(),
                          asset=Asset(href='./{}'.format(output_name), 
                                   media_type=MediaType.GEOTIFF))

    asset = result_item.get_assets()[veg_index.lower()]                                   

    stac_band = extensions.eo.Band.create(name=veg_index.lower(), 
                                               common_name=default_bands[index]['common_name'],
                                               description=default_bands[index]['name'])
    bands.append(stac_band)

    eo_item.set_bands([stac_band], asset=asset)

eo_item.set_bands(bands)

eo_item.apply(bands)  

In [20]:
catalog.add_items([result_item])
    
catalog.normalize_and_save(root_href='./',
                           catalog_type=CatalogType.SELF_CONTAINED)